## Folium maps with Google earth engine

In [1]:
import ee
import folium

In [51]:
# need to sign-up for google earth engine first
ee.Initialize()

In [40]:
ee.Authenticate()

Enter verification code: 4/uQGoeLWU9Pyh2y6aZny9YRFoL_GSk-ZfkExfjG9zy4gxIL1VIyXzGus

Successfully saved authorization token.


In [101]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Fetch an elevation model.
dem1 = ee.Image('USGS/SRTMGL1_003')
dem2 = ee.Image("CSP/ERGo/1_0/Global/ALOS_topoDiversity")
dem3 = ee.Image("CSP/ERGo/1_0/Global/SRTM_mTPI")
dem4 = ee.Image('CGIAR/SRTM90_V4')
dem5 = ee.Image('CSP/ERGo/1_0/Global/ALOS_CHILI')

# Set visualization parameters.
vis_params1 = {
  'min': 0,
  'max': 6000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
vis_params2 = {
  'min': 0,
  'max': 1.0,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
vis_params3 = {
  'min': -500,
  'max': 500,
  'palette': ["0b1eff", "4be450", "fffca4", "ffa011", "ff0000"]}
vis_params4 = {
  'min': 0,
  'max': 6000,
  'palette': ["0b1eff", "4be450", "fffca4", "ffa011", "ff0000"]}
vis_params5 = {
  'min': 0,
  'max': 255,
  'palette': ["0b1eff", "4be450", "fffca4", "ffa011", "ff0000"]}

# Create a folium map object.
my_map = folium.Map(location=[29.5, 99], zoom_start=6, height=600)

# Add the elevation model to the map object. gt can be used to overlay
my_map.add_ee_layer(dem1.updateMask(dem1.gt(0)), vis_params1, 'DEM1')
my_map.add_ee_layer(dem2.updateMask(dem2.gt(0)), vis_params2, 'DEM2')
my_map.add_ee_layer(dem3.updateMask(dem3.gt(0)), vis_params3, 'DEM3')
my_map.add_ee_layer(dem4.updateMask(dem4.gt(0)), vis_params4, 'DEM4')
my_map.add_ee_layer(dem5.updateMask(dem5.gt(0)), vis_params5, 'DEM5')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

### Fetch data from a map

In [103]:
img.select("elevation")

In [102]:
# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

In [104]:
import pandas as pd
import numpy as np
pd.DataFrame(np.array(samp_data).T, columns=["B3", "B4"])

,B3,B4
0,0.0392,0.1271
1,0.0648,0.1357
2,0.1250,0.1684
3,0.0447,0.1338
4,0.0742,0.1562
...,...,...
330,0.0610,0.2457
331,0.1021,0.1997
332,0.1042,0.3168
333,0.1037,0.1561
